In [1]:
#Import packages
import numpy as np
import pandas as pd 
import statsmodels.api as sm
import scipy.stats as stats
from scipy import stats
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt 
from matplotlib.lines import Line2D
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import random
from math import sqrt
import seaborn as sns
plt.style.use('seaborn')

In [2]:
#Import data set
data = pd.read_csv('data\\kc_house_data.csv')
data.isnull().sum()

id                  0
date                0
price               0
bedrooms            0
bathrooms           0
sqft_living         0
sqft_lot            0
floors              0
waterfront       2376
view               63
condition           0
grade               0
sqft_above          0
sqft_basement       0
yr_built            0
yr_renovated     3842
zipcode             0
lat                 0
long                0
sqft_living15       0
sqft_lot15          0
dtype: int64

In [3]:
#refined dataset and drop unnecessary column data
data.drop(['id', 'date', 'waterfront', 'sqft_above', 'sqft_basement', 'lat', 'long', 'view', 'sqft_living15', 'sqft_lot15', 'yr_renovated'], axis=1, inplace=True)

In [4]:
#Removed duplicates, show completed - Scrub
data.drop_duplicates(inplace=True)

In [5]:
data.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,condition,grade,yr_built,zipcode
0,221900.0,3,1.00,1180,5650,1.0,3,7,1955,98178
1,538000.0,3,2.25,2570,7242,2.0,3,7,1951,98125
2,180000.0,2,1.00,770,10000,1.0,3,6,1933,98028
3,604000.0,4,3.00,1960,5000,1.0,5,7,1965,98136
4,510000.0,3,2.00,1680,8080,1.0,3,8,1987,98074


In [6]:
#create the column data
continuous = ['price', 'bedrooms', 'bathrooms', 'sqft_living']
categoricals = ['sqft_lot', 'floors', 'condition', 'grade', 'yr_built', 'zipcode']
data_cont = data[continuous]

In [7]:
# log features
log_names = [f'{column}_log' for column in data_cont.columns]
data_log = np.log(data_cont)
data_log.columns = log_names

In [8]:
# normalize continued features
def normalize(feature):
    return (feature - feature.mean()) / feature.std()
data_log_norm = data_log.apply(normalize)

In [9]:
pd.get_dummies(data)

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,condition,grade,yr_built,zipcode
0,221900.0,3,1.00,1180,5650,1.0,3,7,1955,98178
1,538000.0,3,2.25,2570,7242,2.0,3,7,1951,98125
2,180000.0,2,1.00,770,10000,1.0,3,6,1933,98028
3,604000.0,4,3.00,1960,5000,1.0,5,7,1965,98136
4,510000.0,3,2.00,1680,8080,1.0,3,8,1987,98074
...,...,...,...,...,...,...,...,...,...,...
21592,360000.0,3,2.50,1530,1131,3.0,3,8,2009,98103
21593,400000.0,4,2.50,2310,5813,2.0,3,8,2014,98146
21594,402101.0,2,0.75,1020,1350,2.0,3,7,2009,98144
21595,400000.0,3,2.50,1600,2388,2.0,3,8,2004,98027


In [10]:
data_ohe = pd.get_dummies(data[categoricals], drop_first=True)

In [11]:
preprocessed = pd.concat([data_log_norm, data_ohe], axis=1)
preprocessed.head()

,price_log,bedrooms_log,bathrooms_log,sqft_living_log,sqft_lot,floors,condition,grade,yr_built,zipcode
0,-1.401781,-0.279746,-1.726158,-1.125491,5650,1.0,3,7,1955,98178
1,0.279941,-0.279746,0.339706,0.709304,7242,2.0,3,7,1951,98125
2,-1.799163,-1.711578,-1.726158,-2.131715,10000,1.0,3,6,1933,98028
3,0.499673,0.736156,1.072583,0.070604,5000,1.0,5,7,1965,98136
4,0.178449,-0.279746,0.039651,-0.292754,8080,1.0,3,8,1987,98074


In [12]:
X = preprocessed.drop('price_log', axis=1)
y = preprocessed['price_log']

In [13]:
import statsmodels.api as sm
X_int = sm.add_constant(X)
model = sm.OLS(y,X_int).fit()
model.summary()

C:\Users\racar\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              price_log   R-squared:                       0.636
Model:                            OLS   Adj. R-squared:                  0.636
Method:                 Least Squares   F-statistic:                     4189.
Date:                Mon, 30 May 2022   Prob (F-statistic):               0.00
Time:                        18:50:58   Log-Likelihood:                -19725.
No. Observations:               21589   AIC:                         3.947e+04
Df Residuals:                   21579   BIC:                         3.955e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const             -51.7582      8.443     -6.131      0.000     -68.306     -35.210
bedrooms_log       -0.0924      0.006    -16.538      0.000      -0.103      -0.081
bathrooms_log       0.1093      0.007     14.896      0.000       0.095       0.124
sqft_living_log     0.3533      0.009     40.929      0.000       0.336       0.370
sqft_lot          9.31e-08   1.02e-07      0.917      0.359   -1.06e-07    2.92e-07
floors              0.1394      0.010     14.510      0.000       0.121       0.158
condition           0.0784      0.007     11.307      0.000       0.065       0.092
grade               0.4535      0.006     79.472      0.000       0.442       0.465
yr_built           -0.0111      0.000    -56.414      0.000      -0.012      -0.011
zipcode             0.0007   8.47e-05      8.400      0.000       0.001       0.001
==============================================================================
Omnibus:                       74.481   Durbin-Watson:                   1.965
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               99.071
Skew:                           0.021   Prob(JB):                     3.07e-22
Kurtosis:                       3.329   Cond. No.                     2.05e+08
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.05e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [14]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(X, y)

LinearRegression()

In [15]:
# getting the used column names for the df

continuous.remove("price")

used_cols = [*continuous, *categoricals]
used_cols

['bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'condition',
 'grade',
 'yr_built',
 'zipcode']

In [16]:
# creating an empty dataframe for the new row
new_row = pd.DataFrame(columns=used_cols)

In [17]:
# adding the details provided into the empty dataframe
new_row = new_row.append({"bedrooms": 14977,
                          'bathrooms': 2,
                          'sqft_living':2000,
                          'sqft_lot': '3000',
                          'floors': '2', 
                          'condition': '3',
                          'grade': '6',
                          'yr_built': '1976',
                          'zipcode': '92712'},
                          ignore_index=True)

new_row

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,condition,grade,yr_built,zipcode
0,14977,2,2000,3000,2,3,6,1976,92712


In [18]:
# first we'll tackle the continuous columns
new_row_cont = new_row[continuous]

# log features
log_names = [f'{column}_log' for column in new_row_cont.columns]
pd.options.display.float_format = lambda x : '{:.0f}'.format(x) if int(x) == x else '{:,.2f}'.format(x)
new_row_log = np.log(new_row_cont.astype(float)) # won't work unless float
new_row_log.columns = log_names

# normalizing
for col in continuous:
    # normalize using mean and std from overall dataset
    new_row_log[f'{col}_log'] = (new_row_log[f'{col}_log'] - data[col].mean()) / data[col].std()
new_row_log

,bedrooms_log,bathrooms_log,sqft_living_log
0,6.74,-1.85,-2.26


In [19]:
# now time for the categoricals
new_row_cat = new_row[categoricals]

new_row_ohe = pd.DataFrame(columns = data_ohe.columns)

# using complicated for loops to ohe the new row
pd.options.display.float_format = lambda x : '{:.0f}'.format(x) if int(x) == x else '{:,.2f}'.format(x)
ohe_dict = {}
for col_type in new_row_cat.columns:
    col_list = [c for c in new_row_ohe.columns.to_list() if col_type in c]
    for x in col_list:
        if new_row_cat[col_type][0] in x:
            ohe_dict[x] = 1
        else:
            ohe_dict[x] = 0
            
# putting the results in a dataframe
new_row_ohe = new_row_ohe.append(ohe_dict, ignore_index=True)
new_row_ohe

,sqft_lot,floors,condition,grade,yr_built,zipcode
0,0,0,0,0,0,0


In [20]:
# putting together this row's data - both continuous and categorical
new_row_processed = pd.concat([new_row_log, new_row_ohe], axis=1)
new_row_processed

,bedrooms_log,bathrooms_log,sqft_living_log,sqft_lot,floors,condition,grade,yr_built,zipcode
0,6.74,-1.85,-2.26,0,0,0,0,0,0


In [21]:
# now - FINALLY - we can model

new_row_pred_log = linreg.predict(new_row_processed)
new_row_pred_log

array([-53.38040994])

In [22]:
# prediction needs to be scaled and exponentiated
np.exp(new_row_pred_log) * data["price"].std() + data["price"].mean()

array([540308.34675066])